## Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/Shareddrives/Thesis (PS4)/[01] Colab/[04] DR/DB

Mounted at /content/gdrive
/content/gdrive/Shareddrives/Thesis (PS4)/[01] Colab/[04] DR/DB


##Installing/Importing Libraries

In [ ]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 33.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem

In [ ]:
import joblib

In [ ]:
import pandas as pd
from rdkit import DataStructs
from rdkit import Chem
from rdkit.Chem import AllChem

##DB Drugs

In [ ]:
DB_drugs = pd.read_csv('structure links.csv')
DB_drugs

,DrugBank ID,Name,CAS Number,Drug Groups,InChIKey,InChI,SMILES,Formula,KEGG Compound ID,KEGG Drug ID,PubChem Compound ID,PubChem Substance ID,ChEBI ID,ChEMBL ID,HET ID,ChemSpider ID,BindingDB ID
0,DB00006,Bivalirudin,128270-60-0,approved; investigational,OIRCOABEOLEUMC-GEJPAHFPSA-N,InChI=1S/C98H138N24O33/c1-5-52(4)82(96(153)122...,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,C98H138N24O33,NaN,D03136,16129704.0,46507415.0,59173.0,CHEMBL2103749,NaN,10482069.0,50248103.0
1,DB00007,Leuprolide,53714-56-0,approved; investigational,GFIJNRVAKGFPGQ-LIJARHBVSA-N,InChI=1S/C59H84N16O12/c1-6-63-57(86)48-14-10-2...,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,C59H84N16O12,C07612,D08113,NaN,46507635.0,6427.0,CHEMBL1201199,NaN,571356.0,50369395.0
2,DB00014,Goserelin,65807-02-5,approved,BLCLNMBMMGCOAS-URPVMXJPSA-N,InChI=1S/C59H84N18O14/c1-31(2)22-40(49(82)68-3...,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,C59H84N18O14,NaN,D00573,5311128.0,46507336.0,5523.0,CHEMBL1201247,NaN,4470656.0,NaN
3,DB00027,Gramicidin D,1405-97-6,approved,NDAYQJDHGXTBJL-MWWSRJDJSA-N,InChI=1S/C96H135N19O16/c1-50(2)36-71(105-79(11...,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...,C96H135N19O16,NaN,D04369,45267103.0,46507412.0,NaN,CHEMBL557217,NaN,24623445.0,NaN
4,DB00035,Desmopressin,16679-58-6,approved,NFLWUMRGJYTJIN-PNIOQBSNSA-N,InChI=1S/C46H64N14O12S2/c47-35(62)15-14-29-40(...,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...,C46H64N14O12S2,C06944,D00291,NaN,NaN,4450.0,CHEMBL1429,NaN,4470602.0,50205308.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2764,DB17992,Lotilaner,1369852-71-0,approved; vet_approved,HDKWFBCPLKNOCK-SFHVURJKSA-N,InChI=1S/C20H14Cl3F6N3O3S/c1-8-2-13(36-16(8)17...,CC1=C(SC(=C1)C1=NO[C@@](C1)(C1=CC(Cl)=C(Cl)C(C...,C20H14Cl3F6N3O3S,NaN,NaN,NaN,NaN,NaN,CHEMBL3707310,NaN,32699771.0,NaN
2765,DB18002,Lomifylline,10226-54-7,approved,XBEDAMVJWVPVDS-UHFFFAOYSA-N,InChI=1S/C13H18N4O3/c1-9(18)6-4-5-7-17-8-14-11...,CN1C2=C(N(CCCCC(C)=O)C=N2)C(=O)N(C)C1=O,C13H18N4O3,NaN,NaN,NaN,NaN,94702.0,CHEMBL311948,NaN,59040.0,NaN
2766,DB18712,Berdazimer,NaN,approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2767,DB18716,Enmetazobactam,1001404-83-6,approved,HFZITXBUTWITPT-YWVKMMECSA-N,InChI=1S/C11H14N4O5S/c1-11(6-14-4-3-13(2)12-14...,[H][C@@]12CC(=O)N1[C@@H](C([O-])=O)[C@](C)(CN1...,C11H14N4O5S,NaN,NaN,NaN,NaN,NaN,CHEMBL4458276,NaN,68025157.0,NaN


###Data Cleaning

####Duplicate, NA, erroneous smiles

In [ ]:
# Filter out rows with no SMILES
DB_drugs_no_smiles = DB_drugs[DB_drugs.SMILES.notna()]

# Remove duplicate SMILES
DB_drugs_no_duplicates = DB_drugs_no_smiles.drop_duplicates(['SMILES'])
print("Shape of DataFrame after filtering and removing duplicates:", DB_drugs_no_duplicates.shape)

Shape of DataFrame after filtering and removing duplicates: (2616, 17)


In [ ]:
# Function to check if SMILES is valid
def is_valid_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol is not None

In [ ]:
# Filter out rows with valid SMILES
valid_smiles_mask = DB_drugs_no_duplicates['SMILES'].notnull()
DB_drugs_valid_smiles1 = DB_drugs_no_duplicates[valid_smiles_mask]

# Reset the index after filtering
DB_drugs_valid_smiles1.reset_index(drop=True, inplace=True)

In [ ]:
# Filter out erroneous SMILES
DB_drugs_valid_smiles2 = DB_drugs_valid_smiles1[DB_drugs_valid_smiles1['SMILES'].apply(is_valid_smiles)]
DB_drugs_valid_smiles2.reset_index(drop=True, inplace=True)
DB_drugs_valid_smiles2.shape

[09:12:55] Explicit valence for atom # 84 N, 4, is greater than permitted
[09:12:55] SMILES Parse Error: syntax error while parsing: OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1
[09:12:55] SMILES Parse Error: Failed parsing SMILES 'OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1' for input: 'OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1'
[09:12:55] WARNING: not removing hydrogen atom without neighbors
[09:12:55] WARNING: not removing hydrogen atom without neighbors


(2614, 17)

In [ ]:
# Filter out rows with valid SMILES
valid_smiles_mask = DB_drugs_valid_smiles2['SMILES'].notnull()
DB_drugs_valid_smiles3 = DB_drugs_valid_smiles2[valid_smiles_mask]

# Reset the index after filtering
DB_drugs_valid_smiles3.reset_index(drop=True, inplace=True)
DB_drugs_valid_smiles3.shape

(2614, 17)

In [ ]:
# Filter out erroneous SMILES
DB_drugs_valid_smiles4 = DB_drugs_valid_smiles3[DB_drugs_valid_smiles3['SMILES'].apply(is_valid_smiles)]
DB_drugs_valid_smiles4.reset_index(drop=True, inplace=True)
DB_drugs_valid_smiles4.shape

[09:12:56] WARNING: not removing hydrogen atom without neighbors
[09:12:56] WARNING: not removing hydrogen atom without neighbors


(2614, 17)

####convert to canonical smiles


In [ ]:
#convert to canonical smiles
canonical_smiles = []
for smiles in DB_drugs_valid_smiles4['SMILES']:
    if isinstance(smiles, str):  # Check if the value is a string
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            canonical_mol = Chem.MolToSmiles(mol)
            canonical_smiles.append(canonical_mol)
        else:
            canonical_smiles.append(None)
    else:
        canonical_smiles.append(None)  # If it's not a string, append None

# Create a new DataFrame with canonical SMILES
DB_canonicalsmiles = DB_drugs_valid_smiles4.copy()
DB_canonicalsmiles['canonical_smiles'] = canonical_smiles
DB_canonicalsmiles.shape

[09:12:58] WARNING: not removing hydrogen atom without neighbors
[09:12:58] WARNING: not removing hydrogen atom without neighbors


(2614, 18)

In [ ]:
DB_sortedbyname = DB_canonicalsmiles.sort_values(by='Name')
DB_sortedbyname.reset_index(drop=True, inplace=True)
DB_sortedbyname.to_csv('DB_all_details.csv', index=False)

DB_sortedbyname

,DrugBank ID,Name,CAS Number,Drug Groups,InChIKey,InChI,SMILES,Formula,KEGG Compound ID,KEGG Drug ID,PubChem Compound ID,PubChem Substance ID,ChEBI ID,ChEMBL ID,HET ID,ChemSpider ID,BindingDB ID,canonical_smiles
0,DB12537,"1,2-Benzodiazepine",264-60-8,approved; investigational,SVUOLADPCWQTTE-UHFFFAOYSA-N,InChI=1S/C9H8N2/c1-2-6-9-8(4-1)5-3-7-10-11-9/h...,N1N=CC=CC2=CC=CC=C12,C9H8N2,NaN,NaN,134664.0,347828763.0,NaN,CHEMBL4297264,NaN,118687.0,NaN,C1=Cc2ccccc2NN=C1
1,DB14099,"1,2-Distearoyllecithin",4539-70-2,approved; experimental,NRJAVPSFFCBXDT-UHFFFAOYSA-N,InChI=1S/C44H88NO8P/c1-6-8-10-12-14-16-18-20-2...,CCCCCCCCCCCCCCCCCC(=O)OCC(COP([O-])(=O)OCC[N+]...,C44H88NO8P,NaN,NaN,NaN,NaN,NaN,CHEMBL113644,NaN,58655.0,NaN,CCCCCCCCCCCCCCCCCC(=O)OCC(COP(=O)([O-])OCC[N+]...
2,DB14096,"1,2-icosapentoyl-sn-glycero-3-phosphoserine",NaN,approved; experimental,WQVNGJIIARFEJV-XSLFZJRCSA-N,InChI=1S/C47H72NO10P/c1-3-5-7-9-11-13-15-17-19...,[H]C(N)(COP(O)(=O)OCC([H])(COC(=O)CCC\C=C/C\C=...,C47H72NO10P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32699258.0,NaN,CC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCCCC(=O)OC(COC...
3,DB11331,1-Palmitoyl-2-oleoyl-sn-glycero-3-(phospho-rac...,185435-28-3,approved; experimental,PAZGBAOHGQRCBP-DDDNOICHSA-N,InChI=1S/C40H77O10P/c1-3-5-7-9-11-13-15-17-18-...,CCCCCCCCCCCCCCCC(=O)OC[C@H](COP(O)(=O)OCC(O)CO...,C40H77O10P,NaN,NaN,NaN,NaN,34080.0,NaN,NaN,4446626.0,NaN,CCCCCCCC/C=C\CCCCCCCC(=O)O[C@H](COC(=O)CCCCCCC...
4,DB14201,"2,2'-Dibenzothiazyl disulfide",120-78-5,approved; experimental,AFZSMODLJJCVPP-UHFFFAOYSA-N,InChI=1S/C14H8N2S4/c1-3-7-11-9(5-1)15-13(17-11...,S(SC1=NC2=CC=CC=C2S1)C1=NC2=CC=CC=C2S1,C14H8N2S4,NaN,NaN,NaN,NaN,53239.0,CHEMBL508112,NaN,8139.0,50444458.0,c1ccc2sc(SSc3nc4ccccc4s3)nc2c1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609,DB00132,alpha-Linolenic acid,463-40-1,approved; investigational; nutraceutical,DTOSIQBPPRVQHS-PDBXOOCHSA-N,InChI=1S/C18H30O2/c1-2-3-4-5-6-7-8-9-10-11-12-...,CC\C=C/C\C=C/C\C=C/CCCCCCCC(O)=O,C18H30O2,C06427,NaN,5280934.0,46507620.0,27432.0,CHEMBL8739,LNL,4444437.0,50240347.0,CC/C=C\C/C=C\C/C=C\CCCCCCCC(=O)O
2610,DB14001,alpha-Tocopherol succinate,4345-03-3,approved; nutraceutical; vet_approved,IELOKBJPULMYRW-NJQVLOCASA-N,InChI=1S/C33H54O5/c1-22(2)12-9-13-23(3)14-10-1...,[H][C@@](C)(CCCC(C)C)CCC[C@@]([H])(C)CCC[C@]1(...,C33H54O5,NaN,NaN,NaN,NaN,135821.0,CHEMBL81421,NaN,19171.0,NaN,Cc1c(C)c2c(c(C)c1OC(=O)CCC(=O)O)CC[C@@](C)(CCC...
2611,DB02530,gamma-Aminobutyric acid,56-12-2,approved; investigational,BTCSSZJGUNDROE-UHFFFAOYSA-N,"InChI=1S/C4H9NO2/c5-3-1-2-4(6)7/h1-3,5H2,(H,6,7)",NCCCC(O)=O,C4H9NO2,C00334,D00058,119.0,46507841.0,59888.0,CHEMBL96,ABU,116.0,24183.0,NCCCC(=O)O
2612,DB01440,gamma-Hydroxybutyric acid,591-81-1,approved; illicit; investigational,SJZRECIVHVDYJC-UHFFFAOYSA-N,"InChI=1S/C4H8O3/c5-3-1-2-4(6)7/h5H,1-3H2,(H,6,7)",OCCCC(O)=O,C4H8O3,C00989,NaN,3037032.0,46507548.0,30830.0,CHEMBL1342,NaN,9984.0,50023575.0,O=C(O)CCCO


In [ ]:
DB_drugs_smiles_dbid = pd.concat([DB_sortedbyname['canonical_smiles'], DB_sortedbyname['DrugBank ID']], axis=1)
DB_drugs_smiles_dbid

,canonical_smiles,DrugBank ID
0,C1=Cc2ccccc2NN=C1,DB12537
1,CCCCCCCCCCCCCCCCCC(=O)OCC(COP(=O)([O-])OCC[N+]...,DB14099
2,CC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCCCC(=O)OC(COC...,DB14096
3,CCCCCCCC/C=C\CCCCCCCC(=O)O[C@H](COC(=O)CCCCCCC...,DB11331
4,c1ccc2sc(SSc3nc4ccccc4s3)nc2c1,DB14201
...,...,...
2609,CC/C=C\C/C=C\C/C=C\CCCCCCCC(=O)O,DB00132
2610,Cc1c(C)c2c(c(C)c1OC(=O)CCC(=O)O)CC[C@@](C)(CCC...,DB14001
2611,NCCCC(=O)O,DB02530
2612,O=C(O)CCCO,DB01440


In [ ]:
#Do not run if smi already saved
DB_drugs_smiles_dbid.to_csv('DB_canonicalsmiles9.smi', index=False, header=False)

#DONE
##fingerprint calculation


##Preparing PaDEL


In [ ]:
! pip install padelpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 13.6 MB/s eta 0:00:00


In [ ]:
from padelpy import padeldescriptor

In [ ]:
##Do not run, already downloaded sa folder##

! wget https://github.com/dataprofessor/padel/raw/main/fingerprints_xml.zip
! unzip fingerprints_xml.zip

##Do not run, already downloaded sa folder##

--2024-05-28 09:13:19--  https://github.com/dataprofessor/padel/raw/main/fingerprints_xml.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dataprofessor/padel/main/fingerprints_xml.zip [following]
--2024-05-28 09:13:19--  https://raw.githubusercontent.com/dataprofessor/padel/main/fingerprints_xml.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10871 (11K) [application/zip]
Saving to: ‘fingerprints_xml.zip.1’

fingerprints_xml.zi 100%[===================>]  10.62K  --.-KB/s    in 0.005s  

2024-05-28 09:13:20 (2.28 MB/s) - ‘fingerprints_xml.zip.1’ saved [10871/10871]

Archive:  fingerpri

In [ ]:
import glob
xml_files = glob.glob("*.xml")
xml_files.sort()
xml_files

['AtomPairs2DFingerprintCount.xml',
 'AtomPairs2DFingerprinter.xml',
 'EStateFingerprinter.xml',
 'ExtendedFingerprinter.xml',
 'Fingerprinter.xml',
 'GraphOnlyFingerprinter.xml',
 'KlekotaRothFingerprintCount.xml',
 'KlekotaRothFingerprinter.xml',
 'MACCSFingerprinter.xml',
 'PubchemFingerprinter.xml',
 'SubstructureFingerprintCount.xml',
 'SubstructureFingerprinter.xml']

In [ ]:
FP_list = ['AtomPairs2DCount',
 'AtomPairs2D',
 'EState',
 'CDKextended',
 'CDK',
 'CDKgraphonly',
 'KlekotaRothCount',
 'KlekotaRoth',
 'MACCS',
 'PubChem',
 'SubstructureCount',
 'Substructure']

In [ ]:
fp = dict(zip(FP_list, xml_files))
fp

{'AtomPairs2DCount': 'AtomPairs2DFingerprintCount.xml',
 'AtomPairs2D': 'AtomPairs2DFingerprinter.xml',
 'EState': 'EStateFingerprinter.xml',
 'CDKextended': 'ExtendedFingerprinter.xml',
 'CDK': 'Fingerprinter.xml',
 'CDKgraphonly': 'GraphOnlyFingerprinter.xml',
 'KlekotaRothCount': 'KlekotaRothFingerprintCount.xml',
 'KlekotaRoth': 'KlekotaRothFingerprinter.xml',
 'MACCS': 'MACCSFingerprinter.xml',
 'PubChem': 'PubchemFingerprinter.xml',
 'SubstructureCount': 'SubstructureFingerprintCount.xml',
 'Substructure': 'SubstructureFingerprinter.xml'}

##Calculating Fingerprint

In [ ]:
#DONE DO NOT RUN

In [ ]:
fingerprint = 'KlekotaRothCount'
fingerprint_output_file = ''.join(['DB_', fingerprint,'.csv'])
fingerprint_descriptortypes = fp[fingerprint]

In [ ]:
try:
    # Call padeldescriptor function
    padeldescriptor(mol_dir='DB_canonicalsmiles9.smi',
                    d_file=fingerprint_output_file,
                    descriptortypes=fingerprint_descriptortypes,
                    detectaromaticity=True,
                    standardizenitro=True,
                    standardizetautomers=True,
                    threads=2,
                    removesalt=True,
                    log=True,
                    fingerprints=True)

    print("Descriptors calculation successful.")

except Exception as e:
    print("An error occurred:", str(e))

Descriptors calculation successful.


#Loading the datasets


In [ ]:
KRCfingerprint = pd.read_csv('DB_KlekotaRothCount.csv')
KRCfingerprint

,Name,KRFPC1,KRFPC2,KRFPC3,KRFPC4,KRFPC5,KRFPC6,KRFPC7,KRFPC8,KRFPC9,...,KRFPC4851,KRFPC4852,KRFPC4853,KRFPC4854,KRFPC4855,KRFPC4856,KRFPC4857,KRFPC4858,KRFPC4859,KRFPC4860
0,DB12537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,DB14099,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,DB14096,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,DB11331,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,DB14201,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2605,DB00132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2606,DB14001,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2607,DB02530,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2608,DB01440,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
features_used_in_training_the_model = pd.read_csv('KRC_X_internal_filtered.csv')
# Drop the 'Unnamed: 0' column
features_used_in_training_the_model_no_name = features_used_in_training_the_model.drop(columns=['Unnamed: 0'])
features_used_in_training_the_model_no_name_columns = features_used_in_training_the_model_no_name.columns

In [ ]:
KRC_DB_fingerprint_no_name_column=KRCfingerprint.drop(columns=['Name'])
KRC_DB_fingerprint_no_name_column

,KRFPC1,KRFPC2,KRFPC3,KRFPC4,KRFPC5,KRFPC6,KRFPC7,KRFPC8,KRFPC9,KRFPC10,...,KRFPC4851,KRFPC4852,KRFPC4853,KRFPC4854,KRFPC4855,KRFPC4856,KRFPC4857,KRFPC4858,KRFPC4859,KRFPC4860
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2606,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
KRC_DB_feature_selected = KRC_DB_fingerprint_no_name_column[features_used_in_training_the_model_no_name_columns]
KRC_DB_feature_selected.to_csv('KRC_DB_feature_selected.csv', index=False)
KRC_DB_feature_selected

,KRFPC1,KRFPC18,KRFPC297,KRFPC382,KRFPC444,KRFPC467,KRFPC566,KRFPC605,KRFPC610,KRFPC647,...,KRFPC4065,KRFPC4287,KRFPC4291,KRFPC4293,KRFPC4516,KRFPC4692,KRFPC4752,KRFPC4813,KRFPC4820,KRFPC4824
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,36.0,2.0,1.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.0,0.0,32.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2605,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2606,3.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2607,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2608,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#Predicting IC50 using AD7 model


In [ ]:
DB_alldetails = pd.read_csv('DB_all_details.csv')
DB_alldetails

,DrugBank ID,Name,CAS Number,Drug Groups,InChIKey,InChI,SMILES,Formula,KEGG Compound ID,KEGG Drug ID,PubChem Compound ID,PubChem Substance ID,ChEBI ID,ChEMBL ID,HET ID,ChemSpider ID,BindingDB ID,canonical_smiles
0,DB12537,"1,2-Benzodiazepine",264-60-8,approved; investigational,SVUOLADPCWQTTE-UHFFFAOYSA-N,InChI=1S/C9H8N2/c1-2-6-9-8(4-1)5-3-7-10-11-9/h...,N1N=CC=CC2=CC=CC=C12,C9H8N2,NaN,NaN,134664.0,347828763.0,NaN,CHEMBL4297264,NaN,118687.0,NaN,C1=Cc2ccccc2NN=C1
1,DB14099,"1,2-Distearoyllecithin",4539-70-2,approved; experimental,NRJAVPSFFCBXDT-UHFFFAOYSA-N,InChI=1S/C44H88NO8P/c1-6-8-10-12-14-16-18-20-2...,CCCCCCCCCCCCCCCCCC(=O)OCC(COP([O-])(=O)OCC[N+]...,C44H88NO8P,NaN,NaN,NaN,NaN,NaN,CHEMBL113644,NaN,58655.0,NaN,CCCCCCCCCCCCCCCCCC(=O)OCC(COP(=O)([O-])OCC[N+]...
2,DB14096,"1,2-icosapentoyl-sn-glycero-3-phosphoserine",NaN,approved; experimental,WQVNGJIIARFEJV-XSLFZJRCSA-N,InChI=1S/C47H72NO10P/c1-3-5-7-9-11-13-15-17-19...,[H]C(N)(COP(O)(=O)OCC([H])(COC(=O)CCC\C=C/C\C=...,C47H72NO10P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32699258.0,NaN,CC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCCCC(=O)OC(COC...
3,DB11331,1-Palmitoyl-2-oleoyl-sn-glycero-3-(phospho-rac...,185435-28-3,approved; experimental,PAZGBAOHGQRCBP-DDDNOICHSA-N,InChI=1S/C40H77O10P/c1-3-5-7-9-11-13-15-17-18-...,CCCCCCCCCCCCCCCC(=O)OC[C@H](COP(O)(=O)OCC(O)CO...,C40H77O10P,NaN,NaN,NaN,NaN,34080.0,NaN,NaN,4446626.0,NaN,CCCCCCCC/C=C\CCCCCCCC(=O)O[C@H](COC(=O)CCCCCCC...
4,DB14201,"2,2'-Dibenzothiazyl disulfide",120-78-5,approved; experimental,AFZSMODLJJCVPP-UHFFFAOYSA-N,InChI=1S/C14H8N2S4/c1-3-7-11-9(5-1)15-13(17-11...,S(SC1=NC2=CC=CC=C2S1)C1=NC2=CC=CC=C2S1,C14H8N2S4,NaN,NaN,NaN,NaN,53239.0,CHEMBL508112,NaN,8139.0,50444458.0,c1ccc2sc(SSc3nc4ccccc4s3)nc2c1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2605,DB00132,alpha-Linolenic acid,463-40-1,approved; investigational; nutraceutical,DTOSIQBPPRVQHS-PDBXOOCHSA-N,InChI=1S/C18H30O2/c1-2-3-4-5-6-7-8-9-10-11-12-...,CC\C=C/C\C=C/C\C=C/CCCCCCCC(O)=O,C18H30O2,C06427,NaN,5280934.0,46507620.0,27432.0,CHEMBL8739,LNL,4444437.0,50240347.0,CC/C=C\C/C=C\C/C=C\CCCCCCCC(=O)O
2606,DB14001,alpha-Tocopherol succinate,3/3/4345,approved; nutraceutical; vet_approved,IELOKBJPULMYRW-NJQVLOCASA-N,InChI=1S/C33H54O5/c1-22(2)12-9-13-23(3)14-10-1...,[H][C@@](C)(CCCC(C)C)CCC[C@@]([H])(C)CCC[C@]1(...,C33H54O5,NaN,NaN,NaN,NaN,135821.0,CHEMBL81421,NaN,19171.0,NaN,Cc1c(C)c2c(c(C)c1OC(=O)CCC(=O)O)CC[C@@](C)(CCC...
2607,DB02530,gamma-Aminobutyric acid,56-12-2,approved; investigational,BTCSSZJGUNDROE-UHFFFAOYSA-N,"InChI=1S/C4H9NO2/c5-3-1-2-4(6)7/h1-3,5H2,(H,6,7)",NCCCC(O)=O,C4H9NO2,C00334,D00058,119.0,46507841.0,59888.0,CHEMBL96,ABU,116.0,24183.0,NCCCC(=O)O
2608,DB01440,gamma-Hydroxybutyric acid,591-81-1,approved; illicit; investigational,SJZRECIVHVDYJC-UHFFFAOYSA-N,"InChI=1S/C4H8O3/c5-3-1-2-4(6)7/h5H,1-3H2,(H,6,7)",OCCCC(O)=O,C4H8O3,C00989,NaN,3037032.0,46507548.0,30830.0,CHEMBL1342,NaN,9984.0,50023575.0,O=C(O)CCCO


In [ ]:
rf_model7 = joblib.load('AD7_trained_model_KlekotaRothCount.joblib')

In [ ]:
from sklearn.impute import SimpleImputer

# Initialize the imputer
imputer = SimpleImputer(strategy='mean')

# Fit and transform the data
KRC_DB_feature_selected_imputed = imputer.fit_transform(KRC_DB_feature_selected)

# Make predictions
KRC_DB_drugs_pred7 = rf_model7.predict(KRC_DB_feature_selected_imputed)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.6s
[Parallel(n_jobs=2)]: Done 940 out of 940 | elapsed:    0.7s finished


In [ ]:
KRC_DB_drugs_pred_series7 = pd.Series(KRC_DB_drugs_pred7, name='pIC50')
print(KRC_DB_drugs_pred_series7)

0       4.691777
1       5.350246
2       4.647774
3       4.648371
4       4.287303
          ...   
2605    5.012703
2606    5.484823
2607    4.231812
2608    4.593976
2609    4.588599
Name: pIC50, Length: 2610, dtype: float64


In [ ]:
pic50_prediction_result7 = pd.DataFrame({
    'DB ID': DB_alldetails['DrugBank ID'],
    'Name': DB_alldetails['Name'],
    'Smiles': DB_alldetails['canonical_smiles'],
    'Prediction': KRC_DB_drugs_pred7
})
pic50_prediction_result7

,DB ID,Name,Smiles,Prediction
0,DB12537,"1,2-Benzodiazepine",C1=Cc2ccccc2NN=C1,4.691777
1,DB14099,"1,2-Distearoyllecithin",CCCCCCCCCCCCCCCCCC(=O)OCC(COP(=O)([O-])OCC[N+]...,5.350246
2,DB14096,"1,2-icosapentoyl-sn-glycero-3-phosphoserine",CC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCCCC(=O)OC(COC...,4.647774
3,DB11331,1-Palmitoyl-2-oleoyl-sn-glycero-3-(phospho-rac...,CCCCCCCC/C=C\CCCCCCCC(=O)O[C@H](COC(=O)CCCCCCC...,4.648371
4,DB14201,"2,2'-Dibenzothiazyl disulfide",c1ccc2sc(SSc3nc4ccccc4s3)nc2c1,4.287303
...,...,...,...,...
2605,DB00132,alpha-Linolenic acid,CC/C=C\C/C=C\C/C=C\CCCCCCCC(=O)O,5.012703
2606,DB14001,alpha-Tocopherol succinate,Cc1c(C)c2c(c(C)c1OC(=O)CCC(=O)O)CC[C@@](C)(CCC...,5.484823
2607,DB02530,gamma-Aminobutyric acid,NCCCC(=O)O,4.231812
2608,DB01440,gamma-Hydroxybutyric acid,O=C(O)CCCO,4.593976


In [ ]:
##Change filename
pic50_prediction_result7.to_csv('KRC_DB_drugs_prediction_featureselected.csv', index=False)

In [ ]:
pic50_prediction_result_sorted7 = pic50_prediction_result7.sort_values(by='Prediction', ascending=False)
pic50_prediction_result_sorted7

,DB ID,Name,Smiles,Prediction
2555,DB12026,Voxilaprevir,CC[C@@H]1[C@@H]2CN(C(=O)[C@H](C(C)(C)C)NC(=O)O...,6.920725
816,DB11979,Elagolix,COc1cccc(-c2c(C)n(Cc3c(F)cccc3C(F)(F)F)c(=O)n(...,6.863764
1322,DB15673,Lenacapavir,CC(C)(C#Cc1ccc(-c2ccc(Cl)c3c(NS(C)(=O)=O)nn(CC...,6.806504
1328,DB12070,Letermovir,COc1cccc(N2CCN(C3=Nc4c(F)cccc4[C@H](CC(=O)O)N3...,6.765416
819,DB15444,Elexacaftor,Cc1nn(C)cc1S(=O)(=O)NC(=O)c1ccc(-n2ccc(OCC(C)(...,6.760422
...,...,...,...,...
2226,DB11056,Stannous chloride,[Cl-].[Cl-].[Sn+2],3.608713
2016,DB00339,Pyrazinamide,NC(=O)c1cnccn1,3.499046
1259,DB00951,Isoniazid,NNC(=O)c1ccncc1,3.499046
46,DB00173,Adenine,Nc1ncnc2nc[nH]c12,3.499046


In [ ]:
pic50_prediction_result_sorted7.to_csv('KRC_DB_drugs_prediction_featureselected_sorted7.csv', index=False)